### Imports

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import joblib

### Inputs

In [2]:
edges_dataset_df = pd.read_csv("../datasets/edges_dataset.csv")

X = edges_dataset_df[["rssi", "etx", "delay", "busy_fraction"]].values
y = edges_dataset_df["label"].values

In [3]:
# Normalização
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.3, 
    random_state=42, 
    stratify=y
)

# Divisão de Treino e Teste
X_train = torch.tensor(X_train, dtype=torch.float)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float)
y_test = torch.tensor(y_test, dtype=torch.long)

### Model

In [4]:
class GNN(nn.Module):
    def __init__(self, in_dim=4, hidden=16, out_dim=2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, out_dim)
        )
    def forward(self, x):
        return self.net(x)

### Load Model

In [5]:
model = GNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

### Train

In [6]:
print(f"{'Epoch':<8} {'Loss':<10} {'Train Acc':<12} {'Test Acc':<10}")
print(f"{'-'*42}")

for epoch in range(50):
    model.train()
    optimizer.zero_grad()
    out = model(X_train)
    loss = criterion(out, y_train)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        pred_train = out.argmax(dim=1)
        acc_train = (pred_train == y_train).float().mean().item()
        pred_test = model(X_test).argmax(dim=1)
        acc_test = (pred_test == y_test).float().mean().item()
        
        print(f"{epoch:<8} {loss.item():<10.4f} {acc_train:<12.4f} {acc_test:<10.4f}")

Epoch    Loss       Train Acc    Test Acc  
------------------------------------------
0        0.6342     0.6400       0.8208    
10       0.3272     0.9720       0.9733    
20       0.1374     0.9764       0.9767    
30       0.0749     0.9776       0.9779    
40       0.0568     0.9804       0.9807    


## Metrics

### Save

In [7]:
os.makedirs("../models", exist_ok=True)
torch.save(model.state_dict(), "../models/gnn_model.pth")
joblib.dump(scaler, "../models/scaler_model.pkl")

print("Modelo salvo em ../models/gnn_model.pth")
print("Modelo salvo em ../models/scaler_model.pkl")

Modelo salvo em ../models/gnn_model.pth
Modelo salvo em ../models/scaler_model.pkl
